In [1]:
pip install tensorflow keras matplotlib opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install kaggle

In [3]:
import os
from shutil import copyfile

# Tạo thư mục .kaggle nếu chưa có
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)

# Sao chép tệp kaggle.json vào thư mục .kaggle
copyfile('D:/Download-PC/Cats-and-Dogs/kaggle.json', os.path.expanduser('~/.kaggle/kaggle.json'))

# Cấp quyền cho tệp kaggle.json
os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)

In [4]:
import os
os.environ['PATH'] += os.pathsep + os.path.expanduser('~/.local/bin')

In [5]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
import zipfile

# Đường dẫn đúng đến file ZIP
zip_path = './dogs-vs-cats.zip'

# Giải nén tệp
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('./dogs-vs-cats')

In [7]:
import os
import kaggle
import zipfile

# Đảm bảo tệp kaggle.json đã được sao chép vào ~/.kaggle và có quyền truy cập phù hợp
api = kaggle.KaggleApi()
api.authenticate()

# Tải dataset
api.dataset_download_files('salader/dogs-vs-cats', path='.', unzip=False)

# Kiểm tra file ZIP trong thư mục hiện tại
zip_path = './dogs-vs-cats.zip'
if os.path.exists(zip_path):
    print("File tồn tại.")
    
    # Giải nén tệp
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('./dogs-vs-cats')
    print("Giải nén thành công.")
else:
    print("File không tồn tại. Vui lòng kiểm tra lại đường dẫn.")


Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
File tồn tại.
Giải nén thành công.


In [8]:
# Import các thư viện cần thiết
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

In [9]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory='./dogs-vs-cats/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='./dogs-vs-cats/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [10]:
# chuẩn hóa dữ liệu
def process(image, label):
    image = tf.cast(image / 255.0, tf.float32)
    return image, label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [11]:
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, Input

# create CNN model
model = Sequential()

# Thêm lớp Input
model.add(Input(shape=(256, 256, 3)))

model.add(Conv2D(32, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 60, 60, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,193 (56.64 MB)

 Trainable params: 14,847,745 (56.64 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=validation_ds)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3283s 5s/step - accuracy: 0.5445 - loss: 2.4833 - val_accuracy: 0.6382 - val_loss: 0.6549
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3182s 5s/step - accuracy: 0.6078 - loss: 0.6576 - val_accuracy: 0.5524 - val_loss: 0.7189
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3165s 5s/step - accuracy: 0.6465 - loss: 0.6129 - val_accuracy: 0.6872 - val_loss: 0.5737
Epoch 4/10
279/625 ━━━━━━━━━━━━━━━━━━━━ 27:48 5s/step - accuracy: 0.7107 - loss: 0.5550

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], color='red', label='train')
plt.plot(history.history['val_accuracy'], color='blue', label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], color='red', label='train')
plt.plot(history.history['val_loss'], color='blue', label='validation')
plt.legend()
plt.show()

In [ ]:
import cv2

In [ ]:
test_img = cv2.imread('/content/dogs_vs_cats/test/dogs/dog.10072.jpg')

In [ ]:
plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
test_img = cv2.resize(test_img,(256,256))

In [ ]:
test_input = test_img.reshape((1,256,256,3))

In [ ]:
model.predict(test_input)